# **Data-centric AI: Workflows with cleanlab 2.0+**

In this tutorial, we demonstrate how you can easily incorporate the new and improved [cleanlab 2.0](https://github.com/cleanlab/cleanlab) into your ML development workflows to:

- Automatically find label issues lurking in your data.
- Score the label quality of every example in your dataset.
- Train robust models in the presence of label issues.
- Identify overlapping classes that you can merge to make the learning task less ambiguous.
- Generate an overall label health score to track improvements in your labels as you clean your datasets over time.

This tutorial provides an in-depth survey of many possible different ways that cleanlab can be utilized for Data-Centric AI. If you have a different use-case in mind that is not supported, please [tell us about it](https://github.com/cleanlab/cleanlab/issues)!

Other resources to learn cleanlab: [Documentation](https://docs.cleanlab.ai/) | [


## Install dependencies and import them

You can use `pip` to install all packages required for this tutorial as follows:

```ipython3
!pip install sklearn matplotlib
!pip install cleanlab
# Make sure to install the version corresponding to this tutorial
# E.g. if viewing master branch documentation:
#     !pip install git+https://github.com/cleanlab/cleanlab.git
```

In [ ]:
dependencies = ["cleanlab", "sklearn", "matplotlib"]

if "google.colab" in str(get_ipython()):  # Check if it's running in Google Colab
    %pip install cleanlab  # for colab
    cmd = ' '.join([dep for dep in dependencies if dep != "cleanlab"])
    %pip install $cmd
else:
    missing_dependencies = []
    for dependency in dependencies:
        try:
            __import__(dependency)
        except ImportError:
            missing_dependencies.append(dependency)

    if len(missing_dependencies) > 0:
        print("Missing required dependencies:")
        print(*missing_dependencies, sep=", ")
        print("\nPlease install them before running the rest of this notebook.")

In [ ]:
import numpy as np
import cleanlab
from cleanlab.classification import CleanLearning
from cleanlab.benchmarking import noise_generation
from sklearn.linear_model import LogisticRegression
from numpy.random import multivariate_normal
from matplotlib import pyplot as plt


## Create the data used for this tutorial (can skip these details)


In [ ]:
SEED = 0


def make_data(
    means=[[3, 2], [7, 7], [0, 8], [0, 10]],
    covs=[
        [[5, -1.5], [-1.5, 1]],
        [[1, 0.5], [0.5, 4]],
        [[5, 1], [1, 5]],
        [[3, 1], [1, 1]],
    ],
    sizes=[100, 50, 50, 50],
    avg_trace=0.8,
    seed=SEED,  # set to None for non-reproducible randomness
):
    np.random.seed(seed=SEED)

    K = len(means)  # number of classes
    data = []
    labels = []
    test_data = []
    test_labels = []

    for idx in range(K):
        data.append(
            np.random.multivariate_normal(
                mean=means[idx], cov=covs[idx], size=sizes[idx]
            )
        )
        test_data.append(
            np.random.multivariate_normal(
                mean=means[idx], cov=covs[idx], size=sizes[idx]
            )
        )
        labels.append(np.array([idx for i in range(sizes[idx])]))
        test_labels.append(np.array([idx for i in range(sizes[idx])]))
    X_train = np.vstack(data)
    y_train = np.hstack(labels)
    X_test = np.vstack(test_data)
    y_test = np.hstack(test_labels)

    # Compute p(y=k)
    py_true = np.bincount(y_train) / float(
        len(y_train)
    )  # Prior distribution over true labels.

    noise_matrix_true = noise_generation.generate_noise_matrix_from_trace(
        K,
        trace=avg_trace * K,
        py=py_true,
        valid_noise_matrix=True,
        seed=SEED,
    )

    # Generate our noisy labels using the noise_marix.
    s = noise_generation.generate_noisy_labels(y_train, noise_matrix_true)
    s_test = noise_generation.generate_noisy_labels(y_test, noise_matrix_true)
    ps = np.bincount(s) / float(len(s))  # Prior distribution over noisy labels

    return {
        "data": X_train,
        "true_labels": y_train,  # You never get to see these perfect labels.
        "labels": s,  # Instead, you have these labels, which have some errors.
        "test_data": X_test,
        "test_labels": y_test,  # Perfect labels used for "true" measure of model's performance during deployment.
        "noisy_test_labels": s_test,  # With IID train/test split, you'd have these labels, which also have some errors.
        "ps": ps,
        "py_true": py_true,
        "noise_matrix_true": noise_matrix_true,
        "class_names": ["purple", "blue", "seafoam green", "yellow"],
    }


data_dict = make_data()
for key, val in data_dict.items():  # Map data_dict to variables in namespace
    exec(key + "=val")

# Display dataset visually using matplotlib
plt.figure(figsize=(15, 6))
plt.scatter(data[:, 0], data[:, 1], c=labels, s=60)
true_errors = np.where(true_labels != labels)[0]

for i in true_errors:
    plt.plot(
        data[i][0],
        data[i][1],
        "o",
        markerfacecolor="none",
        markeredgecolor="red",
        markersize=14,
        markeredgewidth=2.5,
        alpha=0.3
    )
_ = plt.title("A realistic, messy dataset with 4 classes.", fontsize=25)


The figure above represents a toy dataset we'll use to demonstrate various cleanlab functionality. In this data, the features *X* are 2-dimensional and examples are colored according to their *given* label above. 

However, [like many real-world datasets](https://labelerrors.com/), the given label happens to be incorrect for some of the examples (**circled in red**) in this dataset!

# **Workflow 1:** Use CleanLearning() for everything


In [ ]:
yourFavoriteModel = LogisticRegression(verbose=0, random_state=SEED)

# CleanLearning: Machine Learning with cleaned data (given messy, real-world data)
cl = cleanlab.classification.CleanLearning(yourFavoriteModel, seed=SEED)

# Fit model to messy, real-world data, automatically training on cleaned data.
_ = cl.fit(data, labels)

# See the label quality for every example, which data has issues, and more.
cl.get_label_issues().head()


## Clean Learning = Machine Learning with Cleaned data


In [ ]:
# For comparison, this is how you would have trained your model normally (without Cleanlab)
yourFavoriteModel = LogisticRegression(verbose=0, random_state=SEED)
yourFavoriteModel.fit(data, labels)
print(
    f"Accuracy using yourFavoriteModel: {yourFavoriteModel.score(test_data, test_labels):.0%}"
)

# But CleanLearning can do anything yourFavoriteModel can do, but enhanced.
# For example, CleanLearning gives you predictions (just like yourFavoriteModel)
# but the magic is that CleanLearning was trained as if your data did not have label errors.
print(
    f"Accuracy using yourFavoriteModel (+ CleanLearning): {cl.score(test_data, test_labels):.0%}"
)


Note! *Accuracy* refers to the accuracy with respect to the *true* error-free labels of a test set., i.e. what we actually care about in practice because that's what real-world model performance is based on. If you don't have a clean test set, you can use cleanlab to make one :)

## **Workflow 2:** Use CleanLearning to find_label_issues in one line of code


In [ ]:
# One line of code. Literally.
issues = CleanLearning(yourFavoriteModel, seed=SEED).find_label_issues(data, labels)

issues.head()


### Visualize the twenty examples with lowest label quality to see if Cleanlab works.


In [ ]:
plt.figure(figsize=(15, 6))
plt.scatter(data[:, 0], data[:, 1], c=labels, s=60)
for i in issues["label_quality"].argsort()[:20]:
    plt.plot(
        data[i][0],
        data[i][1],
        "o",
        markerfacecolor="none",
        markeredgecolor="red",
        markersize=14,
        markeredgewidth=2.5,
    )
_ = plt.suptitle("The 20 lowest label quality examples\nerrors found automatically by cleanlab", fontsize=25)
_ = plt.title("The 20 lowest label quality examples\nerrors found automatically by cleanlab", fontsize=25)


Once you have already computed the label issues, you can pass them into `fit()` and it will run much faster (skips label-issue identification step).

In [ ]:
_ = cl.fit(data, labels, label_issues=issues)

## **Workflow 3:** Use cleanlab to find dataset-level and class-level issues

- Did you notice that the yellow and seafoam green class above are overlapping?
- How can a model learn what's ground truth inside the yellow distribution?
- If these two classes were merged, the model can learn more accurately from 3 classes (versus 4).

cleanlab automatically finds data-set level issues like this, in one line of code. Check this out!


In [ ]:
cleanlab.dataset.find_overlapping_classes(
    labels=labels,
    # confident_joint is computed at cl.fit(). Contains magic cleanlab stats.
    confident_joint=cl.confident_joint,
    class_names=class_names,
)


Do the results surprise you? Did you expect the purple and seafoam green to also have so much overlap?

There are two things happening here:

1. The green distribution has huge variance and its difficult to tell if its a label error or if the classes are simply overlapping distributions. Cleanlab actually handles this case and you can read the theory behind cleanlab for overlapping classes here: https://arxiv.org/abs/1705.01936
2. A ton of examples which actually belong to the purple class have been incorrectly labeled as the green class in our dataset. Data can be so sneaky!

### Now let's see what happens if we merge the top two classes


In [ ]:
print(
    f"[Original classes] Accuracy of yourFavoriteModel: {yourFavoriteModel.score(test_data, test_labels):.0%}"
)

merged_labels, merged_test_labels = np.array(labels), np.array(test_labels)

# Merge classes: map all yellow-labeled examples to seafoam green
merged_labels[merged_labels == 3] = 2
merged_test_labels[merged_test_labels == 3] = 2

# Re-run our comparison. Re-run your model on the newly labeled dataset.
yourFavoriteModel2 = LogisticRegression(verbose=0, random_state=SEED)
yourFavoriteModel2.fit(data, merged_labels)
print(
    f"[Modified classes] Accuracy of yourFavoriteModel: {yourFavoriteModel2.score(test_data, merged_test_labels):.0%}"
)

# Re-run CleanLearning as well.
yourFavoriteModel3 = LogisticRegression(verbose=0, random_state=SEED)
cl2 = cleanlab.classification.CleanLearning(yourFavoriteModel, seed=SEED)
cl2.fit(data, merged_labels)
print(
    f"[Modified classes] Accuracy of yourFavoriteModel (+ CleanLearning): {cl2.score(test_data, merged_test_labels):.0%}"
)


While on one hand that's a huge improvement, it's important to remember that choosing among three classes is an easier task than choosing among four classes, so it's not fair to directly compare these numbers.

Instead, the big takeaway is...
if you get to choose your classes, combining overlapping classes can make the learning task easier for your model. But if you have lots of classes, how do you know which ones to merge?? That's when you use `cleanlab.dataset.find_overlapping_classes`.


## **Workflow 4:** ML with noisy labels and random train/test split

If your test and training data were randomly split (IID), then be aware that your test labels are likely noisy too! It is thus important to fix label issues in them before we can trust measures like test accuracy.

In [ ]:
from sklearn.metrics import accuracy_score

# Fit your model on noisily labeled train data
yourFavoriteModel = LogisticRegression(verbose=0, random_state=SEED)
yourFavoriteModel.fit(data, labels)

# Get predicted probabilities for test data (these are out-of-sample)
my_test_pred_probs = yourFavoriteModel.predict_proba(test_data)
my_test_preds = my_test_pred_probs.argmax(axis=1)  # predicted labels

# Find label issues in the test data
issues_test = CleanLearning(yourFavoriteModel, seed=SEED).find_label_issues(labels=noisy_test_labels,
                                                                            pred_probs=my_test_pred_probs)

# You should inspect issues_test and fix issues to ensure high-quality test data labels. 
corrected_test_labels = test_labels  # Here we'll pretend you have done this perfectly :)

# Fit more robust version of model on noisily labeled training data
cl = CleanLearning(yourFavoriteModel, seed=SEED).fit(data, labels)
cl_test_preds = cl.predict(test_data)

print(
    f"Noisy Accuracy (on given test labels) using yourFavoriteModel: {accuracy_score(noisy_test_labels, my_test_preds):.0%}"
)
print(
    f"Noisy Accuracy (on given test labels) using yourFavoriteModel (+ CleanLearning): {accuracy_score(noisy_test_labels, cl_test_preds):.0%}"
)
print(
    f"Actual Accuracy (on corrected test labels) using yourFavoriteModel: {accuracy_score(corrected_test_labels, my_test_preds):.0%}"
)
print(
    f"Actual Accuracy (on corrected test labels) using yourFavoriteModel (+ CleanLearning): {accuracy_score(corrected_test_labels, cl_test_preds):.0%}"
)


## **Workflow 5:** Scoring the quality of each individual label

cleanlab can analyze every label in a dataset and provide a numerical score gauging its overall quality. Low-quality labels indicate examples that should be more closely inspected, perhaps because their given label is incorrect, or simply because they represent an ambiguous edge-case that's worth a second look.

This score can be fairly compared across datasets or across versions of a dataset to track overall dataset quality (a.k.a. *dataset health*) over time.


In [ ]:
# One line of code.
health = cleanlab.dataset.overall_label_health_score(
    labels, confident_joint=cl.confident_joint
)


### Because we know the truth labels (we created this dataset), we can compare with ground truth.


In [ ]:
label_acc = sum(labels != true_labels) / len(labels)
print(f"Percentage of label issues guessed by cleanlab {1 - health:.0%}")
print(
    f"Percentage of (ground truth) label errors): {label_acc:.0%}",
)

offset = (1 - label_acc) - health

print(
    f"\nQuestion: cleanlab seems to be overestimating."
    f" How do we account for this {offset:.0%} difference?"
)
print(
    "Answer: Data points that fall in between two overlapping distributions are often "
    "impossible to label and are counted as issues."
)


## **Workflow(s) 6:** Use count, rank, filter modules directly

- Using these modules directly is intended for more experienced cleanlab users. But once you understand how they work, you can create numerous powerful workflows.
- For these workflows, you **always** need two things:
  1.  out-of-sample predicted probabilities (e.g. computed via cross-validation) 
  2.  labels (can contain label errors and various issues)

#### cleanlab can compute out-of-sample  predicted probabilities for you:


In [ ]:
pred_probs = cleanlab.count.estimate_cv_predicted_probabilities(
    data, labels, clf=yourFavoriteModel, seed=SEED
)
print(f"pred_probs is a {pred_probs.shape} matrix of predicted probabilites")


### Fully characterize label noise (noise matrix, inverse noise matrix, joint, prior of true labels)

Now that we have `pred_probs` and `labels`, advanced users can compute everything in `cleanlab.count`.

- `py: prob(true_label=k)`
  - For all classes K, this is the distribution over the actual true labels (which cleanlab can estimate for you even though you don't have the true labels).
- `noise_matrix: p(noisy|true)`
  - This describes how errors were introduced into your labels. It's a conditional probability matrix with the probability of flipping from the true class to every other class for the given label.
- `inverse noise matrix: p(true|noisy)`
  - This tells you the probability, for every class, that the true label is actually a different class.
- `confident_joint`
  - This is an unnormalized (count-based) estimate of the number of examples in our dataset with each possible (true label, given label) pairing.
- `joint: p(true label, noisy label)`
  - The joint distribution of noisy (given) and true labels is the most useful of all these statistics. From it, you can compute every other statistic listed above. One entry from this matrix can be interpreted as: "The proportion of examples in our dataset whose true label is *i* and given label is *j*".

These five tools fully characterize class-conditional label noise in a dataset.


In [ ]:
(
    py,
    noise_matrix,
    inverse_noise_matrix,
    confident_joint,
) = cleanlab.count.estimate_py_and_noise_matrices_from_probabilities(labels, pred_probs)

# Note: you can also combine the above two lines of code into a single line of code like this
(
    py,
    noise_matrix,
    inverse_noise_matrix,
    confident_joint,
    pred_probs,
) = cleanlab.count.estimate_py_noise_matrices_and_cv_pred_proba(
    data, labels, clf=yourFavoriteModel, seed=SEED
)

# Get the joint distribution of noisy and true labels from the confident joint
# This is the most powerful statistic in machine learning with noisy labels.
joint = cleanlab.count.estimate_joint(
    labels, pred_probs, confident_joint=confident_joint
)
cleanlab.internal.util.print_joint_matrix(joint)
cleanlab.internal.util.print_noise_matrix(noise_matrix)


In some applications, you may have a priori knowledge regarding some of these quantities. In this case, you can pass them directly into cleanlab which may be able to leverage this information to better identify label issues.


In [ ]:
cl3 = cleanlab.classification.CleanLearning(yourFavoriteModel, seed=SEED)
_ = cl3.fit(data, labels, noise_matrix=noise_matrix_true)  # CleanLearning with a prioiri known noise_matrix

## **Workflow 7:** Get indices of examples with label issues rank-ordered by label quality score

In [ ]:
from sklearn.model_selection import cross_val_predict

# Get out of sample predicted probabilities via cross-validation.
# Here we demonstrate the use of sklearn cross_val_predict as another option to get cross-validated predicted probabilities
cv_pred_probs = cross_val_predict(
    estimator=yourFavoriteModel, X=data, y=labels, cv=3, method="predict_proba"
)

# Find label issues
label_issues_indices = cleanlab.filter.find_label_issues(
    labels=labels,
    pred_probs=cv_pred_probs,
    filter_by="both", # 5 available filter_by options
    return_indices_ranked_by="self_confidence",  # 3 available label quality scoring options for rank ordering
    rank_by_kwargs={
        "adjust_pred_probs": True
    },  # adjust predicted probabilities (see docstring for more details)
)

# Return dataset indices of examples with label issues
label_issues_indices



### Again, we can visualize the twenty examples with lowest label quality to see if Cleanlab works.

In [ ]:
plt.figure(figsize=(15, 6))
plt.scatter(data[:, 0], data[:, 1], c=labels, s=60)
for i in label_issues_indices[:20]: # top 20 from the list of indices
    plt.plot(
        data[i][0],
        data[i][1],
        "o",
        markerfacecolor="none",
        markeredgecolor="red",
        markersize=14,
        markeredgewidth=2.5,
    )
_ = plt.title("The 20 lowest label quality examples", fontsize=25)


## **Workflow 8:** Estimate the number of label issues and use the rank-ordered label quality scores to get the indices of label issues

In [ ]:
# Estimate the number of label issues
label_issues_count = cleanlab.count.num_label_issues(
    labels=labels,
    pred_probs=cv_pred_probs
)

# Get label quality scores
label_quality_scores = cleanlab.rank.get_label_quality_scores(
    labels=labels,
    pred_probs=cv_pred_probs,
    method="self_confidence"
)

# Rank-order by label quality scores and get the top estimated number of label issues
label_issues_indices = np.argsort(label_quality_scores)[:label_issues_count]

TODO:Curtis I think this requires some explanation vs filter and CL find_label_issues, otherwise it seems confusing.


### Again, we can visualize the twenty examples with lowest label quality to see if Cleanlab works.

In [ ]:
plt.figure(figsize=(15, 6))
plt.scatter(data[:, 0], data[:, 1], c=labels, s=60)
for i in label_issues_indices[:20]: # top 20 from the list of indices
    plt.plot(
        data[i][0],
        data[i][1],
        "o",
        markerfacecolor="none",
        markeredgecolor="red",
        markersize=14,
        markeredgewidth=2.5,
    )
_ = plt.title("The 20 lowest label quality examples", fontsize=25)


## **Workflow 9:** Ensembling label quality scores from multiple predictors

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# 3 models in ensemble
model1 = LogisticRegression(penalty="l2", verbose=0, random_state=SEED)
model2 = RandomForestClassifier(max_depth=5, random_state=SEED)
model3 = GradientBoostingClassifier(
    n_estimators=100, learning_rate=1.0, max_depth=3, random_state=SEED
)

# Get cross-validated predicted probabilities from each model
cv_pred_probs_1 = cross_val_predict(
    estimator=model1, X=data, y=labels, cv=3, method="predict_proba"
)

cv_pred_probs_2 = cross_val_predict(
    estimator=model2, X=data, y=labels, cv=3, method="predict_proba"
)

cv_pred_probs_3 = cross_val_predict(
    estimator=model3, X=data, y=labels, cv=3, method="predict_proba"
)

# Get ensemble label quality scores
pred_probs_list = [
    cv_pred_probs_1,
    cv_pred_probs_2,
    cv_pred_probs_3,
]  # list of predicted probabilities from each model

label_quality_scores_best = cleanlab.rank.get_label_quality_ensemble_scores(
    labels=labels, pred_probs_list=pred_probs_list, verbose=False
)

# Alternative approach: create single ensemble predictor and get its pred_probs
cv_pred_probs_ensemble = (cv_pred_probs_1 + cv_pred_probs_2 + cv_pred_probs_3)/3  # uniform aggregation of predictions

# Use this single set of pred_probs to find label issues
label_quality_scores_better = cleanlab.rank.get_label_quality_scores(
    labels=labels, pred_probs=cv_pred_probs_ensemble
)

While ensembling different models' label quality scores (`label_quality_scores_best`) will often be superior to getting label quality scores from a single ensemble predictor (`label_quality_scores_better`), both approaches produce significantly better label quality scores than just using the predictions from a single model. 

## **Bonus:** Try different options of filter_by in find_label_issues() and evaluate precision/recall of detecting true label issues 

In [ ]:
from sklearn.metrics import precision_score, recall_score
import pandas as pd

yourFavoriteModel = LogisticRegression(verbose=0, random_state=SEED)

# get cross-validated predicted probabilities
# here we demonstrate the use of sklearn cross_val_predict as another option to get cross-validated predicted probabilities
cv_pred_probs = cross_val_predict(
    estimator=yourFavoriteModel, X=data, y=labels, cv=3, method="predict_proba"
)

# ground truth label issues to use for evaluating different filter_by options
true_label_issues = (true_labels != labels)

# find label issues with different filter_by options
filter_by_list = [
    "prune_by_noise_rate",
    "prune_by_class",
    "both",
    "confident_learning",
    "predicted_neq_given",
]

results = []

for filter_by in filter_by_list:

    # find label issues
    label_issues = cleanlab.filter.find_label_issues(
        labels=labels, 
        pred_probs=cv_pred_probs, 
        filter_by=filter_by
    )
    
    precision = precision_score(true_label_issues, label_issues)
    recall = recall_score(true_label_issues, label_issues)
    
    result = {
        "filter_by": filter_by,
        "precision": precision,
        "recall": recall
    }
    
    results.append(result)

# summary of results
pd.DataFrame(results)

# TODO -- STUFF BELOW HERE IS JUST NOTES TO HELP YOU WITH THE TODO ABOVE AND IS NOT POLISHED AND SHOULD BE DELETED AFTER YOU USE WHAT YOU NEED!


In [ ]:
pred = pyx.argmax(axis=1)
print("acc:", sum(pred == data["y_train"]) / len(pred))
print("label noise:", sum(data["y_train"] != data["s"]) / len(data["s"]))

# Cleanlab method to find label errors
# label_issues_mask = cleanlab.filter.find_label_issues(data['s'], pyx)

# For simplicity of tutorial, let's use this simple method to find label errors
label_issues_mask = pred != data["s"]


In [ ]:
print("rank label errors")
top = 20
for method in ["normalized_margin", "self_confidence"]:
    idx = cleanlab.rank.order_label_issues(
        label_issues_mask, pyx, data["s"], sorted_index_method=method
    )[:top]
    print(idx)
    plt.figure(figsize=(10, 6))
    for i in idx:
        plt.scatter(
            data["X_train"][:, 0],
            data["X_train"][:, 1],
            c=data["s"],
        )
        plt.plot(
            data["X_train"][i][0],
            data["X_train"][i][1],
            "o",
            markerfacecolor="none",
            markeredgecolor="red",
            markersize=12,
            markeredgewidth=2,
        )
    plt.title("Method: " + method, fontsize=20)
    plt.show()


In [ ]:
# cleanlab.rank.get_normalized_margin_for_each_label(pyx, data['s'])


In [ ]:
print("rank all data")
for method in ["normalized_margin", "self_confidence"]:
    rnk = eval("cleanlab.rank.get_" + method + "_for_each_label(pyx, data['s'])")
    idx = np.argsort(rnk)[:top]
    print(idx)
    plt.figure(figsize=(10, 6))
    for i in idx:
        plt.scatter(
            data["X_train"][:, 0],
            data["X_train"][:, 1],
            c=data["s"],
        )
        plt.plot(
            data["X_train"][i][0],
            data["X_train"][i][1],
            "o",
            markerfacecolor="none",
            markeredgecolor="red",
            markersize=12,
            markeredgewidth=2,
        )
    plt.title("Method: " + method, fontsize=20)
    plt.show()


In [ ]:
print("This should throw an error.")
idx = cleanlab.rank.order_label_issues(
    label_issues_mask, pyx, data["s"], sorted_index_method="something_else"
)


# find_label_issues() supports 5 (instead of 3) methods in Cleanlab 2.0


In [ ]:
cleanlab.filter.find_label_issues()


In [ ]:
true_errors = data["y_train"] != data["s"]
for method in [
    "prune_by_noise_rate",
    "prune_by_class",
    "both",
    "confident_learning",
    "predicted_neq_given",
]:

    label_issues_mask = find_label_issues(data["s"], pyx, filter_by=method)

    err_detection_acc = sum(true_errors == label_issues_mask) / len(true_errors)
    precision = sum((true_errors.astype(int) + label_issues_mask) == 2) / sum(
        label_issues_mask
    )
    recall = sum((true_errors.astype(int) + label_issues_mask) == 2) / sum(true_errors)

    print("Label Error Detection Method:\t", method)
    print("Num errors guessed:\t\t", sum(label_issues_mask))
    print("Actual number of errors:\t", sum(true_errors))
    print("Precision of error detection:\t {:.1%}".format(err_detection_acc))
    print(
        "Precision (Correctly found errors / total errors guessed):\t"
        "{:.1%}".format(precision)
    )
    print(
        "Recall (Correctly found errors / total number errors):\t"
        "{:.1%}".format(recall)
    )
    print()


## Try setting frac_noise == 0.5


In [ ]:
true_errors = data["y_train"] != data["s"]
for method in [
    "prune_by_noise_rate",
    "prune_by_class",
    "both",
    "confident_learning",
    "predicted_neq_given",
]:

    label_issues_mask = find_label_issues(
        data["s"], pyx, filter_by=method, frac_noise=0.5
    )

    err_detection_acc = sum(true_errors == label_issues_mask) / len(true_errors)
    precision = sum((true_errors.astype(int) + label_issues_mask) == 2) / sum(
        label_issues_mask
    )
    recall = sum((true_errors.astype(int) + label_issues_mask) == 2) / sum(true_errors)
    print("Label Error Detection Method:\t", method)
    print("Num errors guessed:\t\t", sum(label_issues_mask))
    print("Actual number of errors:\t", sum(true_errors))
    print("Accuracy of error detection:\t {:.1%}".format(err_detection_acc))
    print(
        "Precision (Correctly found errors / total errors guessed):\t"
        "{:.1%}".format(precision)
    )
    print(
        "Recall (Correctly found errors / total number errors):\t"
        "{:.1%}".format(recall)
    )
    print()
